# TDT4225 Excercise 2 - Part 2

In [7]:
from DbConnector import DbConnector
import SqlQueriespt2 as SQ2

## Question 1: How many users, activities, and trackpoints are there in the dataset


In [8]:
db = DbConnector()

count_users_query = "SELECT COUNT(*) FROM User"
count_activities_query = "SELECT COUNT(*) FROM Activity"
count_trackpoints_query = "SELECT COUNT(*) FROM TrackPoint"

SQ2.query_to_db(db, count_users_query)
SQ2.query_to_db(db, count_activities_query)
SQ2.query_to_db(db, count_trackpoints_query)
db.close_connection()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]
[]
[]

-----------------------------------------------
Connection to 8.0.34-0ubuntu0.22.04.1 is closed


>NOTE! We do not count the original dataset here, we only count the part of the dataset that fits the requirements for the excercise. 

## Question 2: Find the average, maximum, and minimum number of trackpoints per user

In [9]:

def statistics_trackpoints_per_user(function:str = "AVG"):
    string = f"""
        WITH 
            query AS (
                        SELECT COUNT(trackpoint_id) AS trackpoint_count, activity_id FROM TrackPoint
                        GROUP BY activity_id
                    ),
            activities AS (
                SELECT user_id, activity_id FROM Activity
                ),
            joined_user AS (SELECT activities.user_id, query.trackpoint_count FROM query JOIN activities ON query.activity_id = activities.activity_id)

            SELECT user_id, {function}(trackpoint_count) FROM joined_user
            GROUP BY user_id
        ;
        """
    return string

db = DbConnector()
SQ2.query_to_db(db, statistics_trackpoints_per_user("AVG"))
SQ2.query_to_db(db, statistics_trackpoints_per_user("MIN"))
SQ2.query_to_db(db, statistics_trackpoints_per_user("MAX"))
db.close_connection



Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]
[]
[]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x1040a5250>>

>NOTE:
> 
>First value in returned tuple is `user_id` second value is the result of the passed function. 
>
> *Still only using DB values*

## Question 3: Find the top 15 users with the highest number of activities

In [10]:
query_top_15_active_users = """
SELECT user_id, COUNT(activity_id) AS result FROM Activity
GROUP BY user_id
ORDER BY result DESC
LIMIT 15
;
"""

db = DbConnector()
SQ2.query_to_db(db, query_top_15_active_users)
db.close_connection

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x1040a4820>>

## Question 4: Find all users who have taken a bus

In [11]:
bus_users_query = "SELECT DISTINCT user_id FROM Activity WHERE transportation_mode='bus'"
db = DbConnector()
SQ2.query_to_db(db, bus_users_query)
db.close_connection


Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x1040a5520>>

## Question 5: List the top 10 users by their amount of different transportation modes


In [12]:
top_10_users_modes_query = """
SELECT user_id, COUNT(DISTINCT transportation_mode) AS mode_count
FROM Activity
GROUP BY user_id
ORDER BY mode_count DESC
LIMIT 10
"""

db = DbConnector()
SQ2.query_to_db(db, top_10_users_modes_query)
db.close_connection

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x1040a5550>>

## Question 6: Find activities that are registered multiple times

In [13]:
duplicate_activities_query = """
SELECT activity_id
FROM Activity
GROUP BY activity_id
HAVING COUNT(activity_id) > 1
"""
db = DbConnector()
SQ2.query_to_db(db, duplicate_activities_query)
db.close_connection

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x1040a5400>>

## Question 7a: Find the number of users that have started an activity in one day and ended the activity the next day


In [14]:

users_start_end_next_day_query = """
WITH result AS (
    SELECT user_id
    FROM Activity
    WHERE DATE(start_date_time) != DATE(end_date_time)
)
SELECT DISTINCT user_id FROM result
;
"""
db = DbConnector()
SQ2.query_to_db(db, users_start_end_next_day_query)
db.close_connection

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x1040a5d30>>

## Question 7b: List the transportation mode, user id, and duration for these activities

In [15]:
q = """
WITH difference_seconds AS (
    SELECT user_id, transportation_mode, TIMESTAMPDIFF(SECOND, start_date_time, end_date_time) AS seconds
    FROM Activity
    WHERE DATE(start_date_time) != DATE(end_date_time)
),

diff_clock AS (
    SELECT 
        user_id, 
        transportation_mode,
        seconds, 
        MOD(seconds, 60) AS seconds_p,
        MOD(seconds, 60*60) AS minutes_p,
        MOD(seconds, 60*60*24) AS hours_p
    FROM
        difference_seconds
) 

SELECT
    user_id,
    transportation_mode,
    CONCAT(
        FLOOR(seconds / 3600 / 24), ' days ',
        FLOOR(hours_p / 3600), ' hours ',
        FLOOR(minutes_p / 60), ' minutes ',
        seconds_p, ' seconds '
    ) AS duration
FROM diff_clock;
"""

db = DbConnector()
SQ2.query_to_db(db, q)
db.close_connection

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

[]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x104098940>>

## Question 9: Find the top 15 users who have gained the most altitude meters

Here we want to remove all invalid datapoints regarding altitude (e.g. -777) and sumarize the total altitude gained. For our purpose iwe believe we also should ignore the negative altitude gains since we then move downwards. To put it like this; If you walk up a mountain and then down again you would not say you have gained an altitude of 0 meters on the trip. You would only count the ascending meters. Thus we do the same. This would probably lead to the user with most airplane travel to be at the top of our ranking. 

```sql
WITH trackpoint_a_cleaner AS( 

    SELECT TrackPoint.*, t2.user_id
    FROM TrackPoint
    JOIN Activity as t2
    ON TrackPoint.activity_id = t2.activity_id
    WHERE altitude != -777
)

SELECT 
    A.user_id,  
    SUM((B.altitude) - (A.altitude) * 0.3048) as altitude_gain
FROM  trackpoint_a_cleaner AS A
JOIN trackpoint_a_cleaner AS B 
    ON(A.trackpoint_id = B.trackpoint_id-1)
WHERE A.activity_id = B.activity_id 
    AND (B.altitude - A.altitude) > 0
GROUP BY user_id
ORDER BY altitude_gain DESC
LIMIT 15;
```
Resulting in query: 

```
+---------+----------------+
| user_id | altitude_gain  |
+---------+----------------+
| 128     | 139415077.9024 |      <------- Takes airplane a lot. 
| 144     |  58499208.3848 |
| 153     |  47142690.7184 |
| 111     |  24241207.1184 |
| 163     |  21519546.3040 |
| 041     |  18172433.7368 |
| 004     |  16746594.0032 |
| 085     |  13334995.5272 |
| 002     |  11986135.4376 |
| 039     |  10640030.8416 |
| 030     |  10227064.8424 |
| 003     |   9684581.5936 |
| 167     |   9124008.9184 |
| 084     |   8343871.6712 |
| 013     |   6994028.2680 |
+---------+----------------+
15 rows in set (1 min 45.98 sec)
```

## Question 10: Find the users that have traveled the longest total distance in one day for each transportation mode.

Getting all possible values for transportation_mode in Activity

```sql
SELECT DISTINCT transportation_mode FROM Activity;
```

Getting all activities with a certain transport
```sql
WITH transport_same_day AS (
    SELECT 
        user_id, 
        activity_id, 
        transportation_mode 
    FROM 
        Activity 
    WHERE 
        transportation_mode != 'NULL' 
        AND DATE(start_date_time) = DATE(end_date_time)
), 

joined AS (
    SELECT 
        lon, 
        lat,
        date_time, 
        transport_same_day.activity_id, 
        transport_same_day.transportation_mode,
        transport_same_day.user_id  
    FROM 
        TrackPoint 
    JOIN 
        transport_same_day 
    ON 
        TrackPoint.activity_id = transport_same_day.activity_id 
)

SELECT 
    user_id, 
    COUNT(*) AS instances_count, 
    DATE(date_time) AS activity_date,
    activity_id
FROM 
    joined
GROUP BY 
    user_id, DATE(date_time), activity_id
ORDER BY 
    user_id DESC
LIMIT 
    50;
```

Third tactic, treat new joined as trackpoints list: 

FINAL VERSION:
```sql
WITH transport_same_day AS (
    SELECT 
        user_id, 
        activity_id, 
        transportation_mode 
    FROM 
        Activity 
    WHERE 
        transportation_mode != 'NULL' 
        AND DATE(start_date_time) = DATE(end_date_time)
), 

joined AS (
    SELECT 
        lon, 
        lat,
        date_time, 
        transport_same_day.activity_id,
        trackpoint_id,
        transport_same_day.user_id,
        transport_same_day.transportation_mode 
    FROM 
        TrackPoint 
    JOIN 
        transport_same_day 
    ON 
        TrackPoint.activity_id = transport_same_day.activity_id

),

distances_per_day AS (
    SELECT 
        SUM(SQRT(POW(A.lon-B.lon, 2)+POW(A.lat-B.lat, 2))*111139) as distance_in_meters, 
        A.activity_id as activity_id, 
        A.user_id as user_id, 
        A.transportation_mode AS transportation_mode,
        DATE(A.date_time) AS date_date
    FROM joined A
    JOIN joined B ON (A.trackpoint_id = B.trackpoint_id -1)
    GROUP BY DATE(date_time), activity_id
),
sumarized_by_day_tm AS(
SELECT 
    user_id, 
    SUM(distance_in_meters) AS total_daily_distance, 
    transportation_mode,
    date_date 
FROM distances_per_day
GROUP BY user_id, date_date, transportation_mode
)
SELECT user_id, transportation_mode, total_daily_distance, date_date
FROM sumarized_by_day_tm
WHERE (total_daily_distance, transportation_mode) 
IN(
    SELECT MAX(total_daily_distance),transportation_mode 
    FROM sumarized_by_day_tm 
    GROUP BY transportation_mode
    )
;
```

OUTPUT OF 
```
+---------+---------------------+----------------------+------------+
| user_id | transportation_mode | total_daily_distance | date_date  |
+---------+---------------------+----------------------+------------+
| 108     | walk                |    29273.53825036155 | 2007-10-03 |
| 139     | walk                |    29273.53825036155 | 2007-10-03 |
| 128     | bike                |    73143.37846558148 | 2008-06-28 |
| 062     | run                 |   40.082311659663425 | 2008-09-02 |
| 062     | train               |    350357.8121884449 | 2008-09-02 |
| 128     | taxi                |   41157.635837726746 | 2008-09-30 |
| 128     | car                 |   520868.34688762604 | 2008-10-02 |
| 128     | subway              |    35760.66025889879 | 2008-10-31 |
| 128     | boat                |    69989.41164522138 | 2008-11-22 |
| 128     | bus                 |   221259.96968129047 | 2009-01-20 |
| 128     | airplane            |    44462819.63427694 | 2009-03-06 |
+---------+---------------------+----------------------+------------+
```

Here we hace two users with the exact same amount of total daily distance of walk. Also on the same date :O 

```
mysql> SELECT *      FROM Activity     WHERE Activity.transportation_mode != 'NULL'      AND user_id = '108';
+-------------+---------+---------------------+---------------------+---------------------+
| activity_id | user_id | transportation_mode | start_date_time     | end_date_time       |
+-------------+---------+---------------------+---------------------+---------------------+
|       12663 | 108     | walk                | 2007-10-02 14:25:15 | 2007-10-02 14:45:18 |     <------!!!
|       12665 | 108     | bike                | 2007-10-09 00:26:41 | 2007-10-09 00:40:32 |     <------!!!
|       12671 | 108     | walk                | 2007-10-03 07:58:04 | 2007-10-03 11:54:19 |     <------!!!
+-------------+---------+---------------------+---------------------+---------------------+
3 rows in set (0.00 sec)

mysql> SELECT *      FROM Activity     WHERE Activity.transportation_mode != 'NULL'      AND user_id = '139';
+-------------+---------+---------------------+---------------------+---------------------+
| activity_id | user_id | transportation_mode | start_date_time     | end_date_time       |
+-------------+---------+---------------------+---------------------+---------------------+
|        9102 | 139     | walk                | 2007-10-02 14:25:15 | 2007-10-02 14:45:18 |     <------!!!
|        9105 | 139     | bike                | 2007-09-20 14:43:58 | 2007-09-20 14:55:00 |
|        9106 | 139     | bike                | 2007-10-09 00:26:41 | 2007-10-09 00:40:32 |     <------!!!
|        9112 | 139     | bike                | 2007-09-14 00:42:03 | 2007-09-14 00:45:21 |
|        9117 | 139     | walk                | 2007-10-03 07:58:04 | 2007-10-03 11:54:19 |     <------!!!
|        9119 | 139     | bike                | 2007-09-19 00:35:51 | 2007-09-19 00:43:45 |
+-------------+---------+---------------------+---------------------+---------------------+
6 rows in set (0.00 sec)
```
A simple analysis shows that we have 3 suspiciously similar entries marked with `<------!!!`
```sql
WITH 108_139_acts AS(
    SELECT * FROM Activity WHERE user_id = '108' or user_id = '139'
),

merged AS (
    SELECT A.*, B.user_id 
    FROM TrackPoint as A
    JOIN 108_139_acts as B
    ON A.activity_id = B.activity_id
)

SELECT A.activity_id, A.date_time, A.lon, A.lat, A.user_id, B.activity_id, B.date_time, B.lon, B.lat, B.user_id
FROM (
    SELECT *
    FROM merged
    WHERE user_id = '108') as A
JOIN (
    SELECT *
    FROM merged
    WHERE user_id = '139') as B
ON A.date_time = B.date_time
WHERE A.lon = B.lon 
AND A.lat = B.lat; 

```
Resulting in : 

![image](./img_notebook/suspicious10_1.png)
...
![image](./img_notebook/suspicious10_2.png)

Here we see that the users have 484 duplicate records.
Advised action would be to drop some of the duplicate actions or do a deeper analysis to see if one user can be deleted in its entirety. 


## Question 11: Find all users who have invalid activities, and the number of invalid activities per user
- An invalid activity is defined as an activity with consecutive trackpoints where the timestamps deviate with at least 5 minutes.


First we test if we can find the difference in time between the different trackpoints in any one given activity: 

```sql
SELECT 
    A.activity_id,
    TIMESTAMPDIFF(SECOND, A.date_time, B.date_time) as sec_dif
FROM TrackPoint A
JOIN TrackPoint B
ON (A.trackpoint_id = B.trackpoint_id -1)
WHERE A.activity_id = 1 AND B.activity_id = 1
;
```
Below we first see all tracpoints associated with activity_id 1 and then see the results from the query above. 
```
+---------------+-------------+------------------+------------------+----------+---------------------+
| trackpoint_id | activity_id | lat              | lon              | altitude | date_time           |
+---------------+-------------+------------------+------------------+----------+---------------------+
|             1 |           1 |         39.96335 |         116.3307 |      105 | 2007-10-22 01:47:04 |
|             2 |           1 |          39.9659 | 116.314716666667 |      105 | 2007-10-22 01:47:40 |
|             3 |           1 | 39.9659166666667 |        116.31565 |      105 | 2007-10-22 01:48:25 |
|             4 |           1 |          39.9665 | 116.315866666667 |      105 | 2007-10-22 01:49:25 |
|             5 |           1 | 39.9669166666667 | 116.314716666667 |      125 | 2007-10-22 01:49:46 |
|             6 |           1 | 39.9679666666667 | 116.314283333333 |      217 | 2007-10-22 01:51:19 |
|             7 |           1 |          39.9681 | 116.313866666667 |      285 | 2007-10-22 01:56:45 |
+---------------+-------------+------------------+------------------+----------+---------------------+
7 rows in set (0.00 sec)

+-------------+---------+
| activity_id | sec_dif |
+-------------+---------+
|           1 |      36 |
|           1 |      45 |
|           1 |      60 |
|           1 |      21 |
|           1 |      93 |
|           1 |     326 |
+-------------+---------+
6 rows in set (0.00 sec)

```
We can verify that the difference between each trackpoint is correct. 

Now we want to test if we can only get the differences that are above 60 seconds

```sql
SELECT *
FROM(

    SELECT 
        A.activity_id,
        A.trackpoint_id as track_id_A,
        B.trackpoint_id as track_id_B,
        TIMESTAMPDIFF(SECOND, A.date_time, B.date_time) as sec_dif
    FROM TrackPoint A
    JOIN TrackPoint B
    ON (A.trackpoint_id = B.trackpoint_id -1)
    WHERE A.activity_id = 1 AND B.activity_id = 1
) as dif_table
WHERE sec_dif > 60
;
```
This resulted in: 
```
+-------------+------------+------------+---------+
| activity_id | track_id_A | track_id_B | sec_dif |
+-------------+------------+------------+---------+
|           1 |          5 |          6 |      93 |
|           1 |          6 |          7 |     326 |
+-------------+------------+------------+---------+
2 rows in set (0.00 sec)
```

Now we want to run this on all trackpoints, resulting in a list with ONLY the activity id. We don really need to keep track of the different trackpoint differences after we've chosen only those activities with a difference above our threshold. Thus we only need to set the `WHERE statement to A.activity_id = B.activity_id` and select distinct activites from the resulting table

```sql
WITH invalid_act AS(
    SELECT *
    FROM(

        SELECT 
            A.activity_id,
            A.trackpoint_id as track_id_A,
            B.trackpoint_id as track_id_B,
            TIMESTAMPDIFF(SECOND, A.date_time, B.date_time) as sec_dif
        FROM TrackPoint A
        JOIN TrackPoint B
        ON (A.trackpoint_id = B.trackpoint_id -1)
        WHERE A.activity_id = B.activity_id
    ) as dif_table
    WHERE sec_dif > 300 /* 5 minutes */
)

SELECT DISTINCT activity_id
FROM invalid_act
;
```

Resulting in many rows, here are the last few: 
```
|       16028 |
|       16029 |
|       16034 |
|       16035 |
|       16036 |
|       16038 |
|       16040 |
|       16044 |
|       16046 |
|       16047 |
|       16048 |
+-------------+
7001 rows in set (1 min 12.71 sec)
```

Now we only need to join this with the activity table to get the user_id and count instances to get the final result. 

```sql
WITH invalid_act AS(
    SELECT *
    FROM(

        SELECT 
            A.activity_id,
            A.trackpoint_id as track_id_A,
            B.trackpoint_id as track_id_B,
            TIMESTAMPDIFF(SECOND, A.date_time, B.date_time) as sec_dif
        FROM TrackPoint A
        JOIN TrackPoint B
        ON (A.trackpoint_id = B.trackpoint_id -1)
        WHERE A.activity_id = B.activity_id
    ) as dif_table
    WHERE sec_dif > 300 /* 5 minutes */
),

dist_acts AS (
SELECT DISTINCT activity_id
FROM invalid_act
)

SELECT 
    Activity.user_id, 
    COUNT(dist_acts.activity_id) as N_invalid_activities
FROM dist_acts
JOIN Activity
ON Activity.activity_id = dist_acts.activity_id
GROUP BY user_id
;
```

```
+---------+----------------------+
| user_id | N_invalid_activities |
+---------+----------------------+
| 000     |                  101 |
| 001     |                   45 |
| 002     |                   98 |
| 003     |                  179 |
| 004     |                  219 |
| 005     |                   44 |
| 006     |                   17 |
| 007     |                   30 |
| 008     |                   16 |
| 009     |                   31 |
| 010     |                   50 |
| 011     |                   32 |
| 012     |                   43 |
| 013     |                   29 |
| 014     |                  118 |
| 015     |                   46 |
| 016     |                   20 |
| 017     |                  129 |
| 018     |                   27 |
| 019     |                   31 |
| 020     |                   20 |
| 021     |                    7 |
| 022     |                   55 |
| 023     |                   11 |
| 024     |                   27 |
| 025     |                  263 |
| 026     |                   18 |
| 027     |                    2 |
| 028     |                   36 |
| 029     |                   25 |
| 030     |                  112 |
| 031     |                    3 |
| 032     |                   12 |
| 033     |                    2 |
| 034     |                   88 |
| 035     |                   23 |
| 036     |                   34 |
| 037     |                  100 |
| 038     |                   58 |
| 039     |                  147 |
| 040     |                   17 |
| 041     |                  201 |
| 042     |                   54 |
| 043     |                   21 |
| 044     |                   31 |
| 045     |                    7 |
| 046     |                   13 |
| 047     |                    6 |
| 048     |                    1 |
| 050     |                    8 |
| 051     |                   36 |
| 052     |                   44 |
| 053     |                    7 |
| 054     |                    2 |
| 055     |                   15 |
| 056     |                    7 |
| 057     |                   16 |
| 058     |                   13 |
| 059     |                    5 |
| 060     |                    1 |
| 061     |                   12 |
| 062     |                  248 |
| 063     |                    8 |
| 064     |                    7 |
| 065     |                   25 |
| 066     |                    6 |
| 067     |                   33 |
| 068     |                  139 |
| 069     |                    6 |
| 070     |                    5 |
| 071     |                   27 |
| 072     |                    2 |
| 073     |                   17 |
| 074     |                   19 |
| 075     |                    6 |
| 076     |                    8 |
| 077     |                    3 |
| 078     |                   19 |
| 079     |                    2 |
| 080     |                    6 |
| 081     |                   16 |
| 082     |                   27 |
| 083     |                   15 |
| 084     |                   99 |
| 085     |                  182 |
| 086     |                    5 |
| 087     |                    3 |
| 088     |                   11 |
| 089     |                   40 |
| 090     |                    3 |
| 091     |                   63 |
| 092     |                  101 |
| 093     |                    4 |
| 094     |                   16 |
| 095     |                    4 |
| 096     |                   35 |
| 097     |                   14 |
| 098     |                    5 |
| 099     |                   11 |
| 100     |                    3 |
| 101     |                   46 |
| 102     |                   13 |
| 103     |                   24 |
| 104     |                   97 |
| 105     |                    9 |
| 106     |                    3 |
| 107     |                    1 |
| 108     |                    5 |
| 109     |                    3 |
| 110     |                   17 |
| 111     |                   26 |
| 112     |                   66 |
| 113     |                    1 |
| 114     |                    3 |
| 115     |                   58 |
| 117     |                    3 |
| 118     |                    3 |
| 119     |                   22 |
| 121     |                    4 |
| 122     |                    6 |
| 123     |                    3 |
| 124     |                    4 |
| 125     |                   25 |
| 126     |                  104 |
| 127     |                    4 |
| 128     |                  719 |
| 129     |                    6 |
| 130     |                    8 |
| 131     |                   10 |
| 132     |                    3 |
| 133     |                    4 |
| 134     |                   31 |
| 135     |                    5 |
| 136     |                    6 |
| 138     |                   10 |
| 139     |                   12 |
| 140     |                   86 |
| 141     |                    1 |
| 142     |                   52 |
| 144     |                  157 |
| 145     |                    5 |
| 146     |                    7 |
| 147     |                   30 |
| 150     |                   16 |
| 151     |                    1 |
| 152     |                    2 |
| 153     |                  556 |
| 154     |                   14 |
| 155     |                   30 |
| 157     |                    9 |
| 158     |                    9 |
| 159     |                    5 |
| 161     |                    7 |
| 162     |                    9 |
| 163     |                  232 |
| 164     |                    6 |
| 165     |                    2 |
| 166     |                    2 |
| 167     |                  134 |
| 168     |                   19 |
| 169     |                    9 |
| 170     |                    2 |
| 171     |                    3 |
| 172     |                    9 |
| 173     |                    5 |
| 174     |                   54 |
| 175     |                    4 |
| 176     |                    8 |
| 179     |                   28 |
| 180     |                    2 |
| 181     |                   14 |
+---------+----------------------+
171 rows in set (2 min 3.58 sec)
```

## Question 12: Find all users who have registered transportation_mode and their most used transportation_mode.
- The answer should be on format (user_id, most_used_transportation_mode) sorted on user_id.
- Some users may have the same number of activities tagged with e.g. walk and car. In this case it is up to you to decide which transportation mode to include in your answer (choose one).
- Do not count the rows where the mode is null.

```SQL
WITH transport_modes AS (
    SELECT 
        user_id, 
        activity_id, 
        transportation_mode 
    FROM 
        Activity 
    WHERE 
        transportation_mode != 'NULL' 
), 

counted_tm AS (
    SELECT 
        user_id, 
        COUNT(*) AS n_used, 
        transportation_mode
    FROM 
        transport_modes
    GROUP BY 
        transportation_mode, user_id
),

max_for_each_user AS (
    SELECT 
        A.user_id, 
        A.n_used AS max_count,
        A.transportation_mode
    FROM 
        counted_tm AS A
    JOIN (
        SELECT 
            user_id, 
            MAX(n_used) AS max_n_used
        FROM 
            counted_tm
        GROUP BY 
            user_id
    ) AS B
    ON 
        A.user_id = B.user_id 
        AND A.n_used = B.max_n_used
)

SELECT max_for_each_user.*
FROM max_for_each_user
WHERE max_for_each_user.transportation_mode = (
    SELECT t2.transportation_mode
    FROM max_for_each_user as t2
    WHERE t2.user_id = max_for_each_user.user_id
    ORDER BY rand()
    LIMIT 1
)
ORDER BY max_for_each_user.user_id
;
```

RESULTS
```
+---------+-----------+---------------------+
| user_id | max_count | transportation_mode |
+---------+-----------+---------------------+
| 010     |         3 | taxi                |
| 020     |        81 | bike                |
| 021     |         1 | walk                |
| 052     |         1 | bus                 |
| 056     |        15 | bike                |
| 058     |         2 | walk                |
| 060     |         1 | walk                |
| 062     |       173 | bus                 |
| 064     |         1 | bike                |
| 065     |        10 | bike                |
| 067     |         1 | walk                |
| 069     |         1 | bike                |
| 073     |        52 | walk                |
| 075     |         1 | walk                |
| 076     |         3 | car                 |
| 078     |        37 | walk                |
| 081     |         4 | bike                |
| 082     |         2 | walk                |
| 084     |         9 | walk                |
| 085     |        18 | walk                |
| 086     |         2 | car                 |
| 087     |         5 | walk                |
| 089     |         7 | car                 |
| 091     |         2 | walk                |
| 092     |         1 | walk                |
| 097     |         9 | bike                |
| 098     |         1 | taxi                |
| 101     |         3 | car                 |
| 102     |         7 | bike                |
| 107     |         1 | walk                |
| 108     |         2 | walk                |
| 111     |         3 | taxi                |
| 112     |        69 | walk                |
| 115     |        80 | car                 |
| 117     |         1 | walk                |
| 125     |         3 | bike                |
| 126     |        13 | bike                |
| 128     |       312 | car                 |
| 136     |         3 | walk                |
| 138     |         2 | bike                |
| 139     |         4 | bike                |
| 144     |         2 | walk                |
| 153     |         5 | walk                |
| 161     |         1 | walk                |
| 163     |        26 | bike                |
| 167     |        31 | bike                |
| 175     |         1 | bus                 |
+---------+-----------+---------------------+
47 rows in set (0.01 sec)
```